# Pandas группировка и агрегация

Считаем тестовый набор данных

In [1]:
import pandas as pd

df = pd.read_csv('./data/StudentsPerformance.csv')

df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


Посчитаем среднее всех score в зависимости от пола студента

In [2]:
df.groupby('gender').mean()

,math score,reading score,writing score
gender,,,
female,63.633205,72.608108,72.467181
male,68.728216,65.473029,63.311203


In [3]:
df.groupby('gender').aggregate({'math score': 'mean', 'reading score': 'mean'})

,math score,reading score
gender,,
female,63.633205,72.608108
male,68.728216,65.473029


In [4]:
df.groupby('gender', as_index=False) \
    .aggregate({'math score': 'mean', 'reading score': 'mean'}) \
    .rename(columns={'math score': 'mean_math_score', 'reading score': 'mean_reading_score'})

,gender,mean_math_score,mean_reading_score
0,female,63.633205,72.608108
1,male,68.728216,65.473029


Сгруппируем студентов по нескольким переменным: по полу и по группе

In [5]:
mean_scores = df.groupby(['gender', 'race/ethnicity']) \
    .aggregate({'math score': 'mean', 'reading score': 'mean'}) \
    .rename(columns={'math score': 'mean_math_score', 'reading score': 'mean_reading_score'})
mean_scores

mean_math_score  mean_reading_score
gender race/ethnicity                                     
female group A               58.527778           69.000000
       group B               61.403846           71.076923
       group C               62.033333           71.944444
       group D               65.248062           74.046512
       group E               70.811594           75.840580
male   group A               63.735849           61.735849
       group B               65.930233           62.848837
       group C               67.611511           65.424460
       group D               69.413534           66.135338
       group E               76.746479           70.295775

Работа с multiple индексами

In [6]:
mean_scores.index

MultiIndex([('female', 'group A'),
            ('female', 'group B'),
            ('female', 'group C'),
            ('female', 'group D'),
            ('female', 'group E'),
            (  'male', 'group A'),
            (  'male', 'group B'),
            (  'male', 'group C'),
            (  'male', 'group D'),
            (  'male', 'group E')],
           names=['gender', 'race/ethnicity'])

In [7]:
mean_scores.loc[[('male', 'group C'), ('male', 'group D')]]

mean_math_score  mean_reading_score
gender race/ethnicity                                     
male   group C               67.611511           65.424460
       group D               69.413534           66.135338

Выведем количество оценок на пересечении групп

In [8]:
df.groupby(['gender', 'race/ethnicity'])['math score'].nunique()

gender  race/ethnicity
female  group A           29
        group B           51
        group C           59
        group D           53
        group E           44
male    group A           38
        group B           43
        group C           56
        group D           49
        group E           38
Name: math score, dtype: int64

Найдём топ 5 студентов женского и мужского пола по оценке math score

In [9]:
df.sort_values(['gender', 'math score'], ascending=False).groupby('gender').head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
149,male,group E,associate's degree,free/reduced,completed,100,100,93
623,male,group A,some college,standard,completed,100,96,86
625,male,group D,some college,standard,completed,100,97,99
916,male,group E,bachelor's degree,standard,completed,100,100,100
306,male,group E,some college,standard,completed,99,87,81
451,female,group E,some college,standard,none,100,92,97
458,female,group E,bachelor's degree,standard,none,100,100,100
962,female,group E,associate's degree,standard,none,100,100,100
114,female,group E,bachelor's degree,standard,completed,99,100,100
263,female,group E,high school,standard,none,99,93,90


Добавим новую колонку total score

In [10]:
df['total_score'] = df['math score'] + df['reading score'] + df['writing score']
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,total_score
0,female,group B,bachelor's degree,standard,none,72,72,74,218
1,female,group C,some college,standard,completed,69,90,88,247
2,female,group B,master's degree,standard,none,90,95,93,278
3,male,group A,associate's degree,free/reduced,none,47,57,44,148
4,male,group C,some college,standard,none,76,78,75,229


Сложные преобразования

In [11]:
import numpy as np

df = df.assign(total_score_log=np.log(df.total_score))
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,total_score,total_score_log
0,female,group B,bachelor's degree,standard,none,72,72,74,218,5.384495
1,female,group C,some college,standard,completed,69,90,88,247,5.509388
2,female,group B,master's degree,standard,none,90,95,93,278,5.627621
3,male,group A,associate's degree,free/reduced,none,47,57,44,148,4.997212
4,male,group C,some college,standard,none,76,78,75,229,5.433722


Удаляем колонки

In [12]:
df.drop(['total_score', 'total_score_log'], axis=1)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


### Задание  
Пересчитаем число ног у героев игры Dota2! Сгруппируйте героев из датасэта по числу их ног (колонка legs).

In [13]:
df = pd.read_csv('https://stepik.org/media/attachments/course/4852/dota_hero_stats.csv')

df.groupby('legs').size()

legs
0    11
2    95
4     7
6     3
8     1
dtype: int64

### Задание  
К нам поступили данные из бухгалтерии о заработках Лупы и Пупы за разные задачи!  
Посмотрите у кого из них больше средний заработок в различных категориях (колонка Type).

In [14]:
df = pd.read_csv('https://stepik.org/media/attachments/course/4852/accountancy.csv')

df.groupby(['Executor', 'Type']).Salary.mean().unstack()

Type,A,B,C,D,E,F
Executor,,,,,,
Loopa,58.000000,145.166667,154.333333,137.714286,164.0,238.00
Pupa,160.833333,77.000000,74.500000,146.500000,131.2,136.25


### Задание  
Аспирант Ростислав изучает метаболом водорослей и получил такую табличку.  
В ней он записал вид каждой водоросли, её род (группа, объединяющая близкие виды),  
группа (ещё одно объединение водорослей в крупные фракции) и концентрации анализируемых веществ.  

Помогите Ростиславу найти среднюю концентрацию каждого из веществ в каждом из родов (колонка genus)!

In [15]:
df = pd.read_csv('http://stepik.org/media/attachments/course/4852/algae.csv')

df.groupby('genus').mean()

,sucrose,alanin,citrate,glucose,oleic_acid
genus,,,,,
Ascophyllum,6.825467,0.875429,5.253527,3.414961,2.432526
Cladophora,4.008792,3.997055,5.288311,2.800276,2.263472
Fucus,4.355112,3.566411,4.214878,2.814803,3.367647
Palmaria,0.704580,3.176440,5.573905,3.242090,2.245538
Saccharina,4.183596,3.524207,3.346710,4.980594,4.487252


Пользуясь предыдущими данными, укажите чему равны минимальная,  
средняя и максимальная концентрации аланина (alanin) среди видов рода Fucus.

In [16]:
df.groupby('genus').agg(['min', 'mean', 'max']).loc['Fucus', 'alanin'].round(2)

min     0.35
mean    3.57
max     6.64
Name: Fucus, dtype: float64